<a href="https://colab.research.google.com/github/lakesidetech/machine_learning/blob/main/Copy_of_Employee_attrition_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning Model for Predicting Employee Attrition
We are building a simple model using the ***employee_attrition_test.csv*** data set for predicting an employee attrition. This project is organized as follows: (a) data preprocessing and variable selection; (b) basic regression model; (c) hyper-parameters tuning; and (d) techniques for dimensionality reduction.

Author: Ruth A. Amolo & Nancy Wachira Date: 5/15/2021


# import libraries

In [ ]:
#data processing
import pandas as pd

# linear algebra
import numpy as np

# data visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas import read_csv

#Algorithms
from sklearn import linear_model
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



**1. Read dataset and display columns¶**


In [ ]:
#hr_attrition_test_df=pd.read_csv('/content/employee_attrition_test.csv')
#hr_attrition_train_df=pd.read_csv('/content/employee_attrition_train.csv')

hr_attrition_test_df=pd.read_csv('/content/employee_attrition_test.csv')
hr_attrition_train_df=pd.read_csv('/content/employee_attrition_train.csv')

In [ ]:
test_data = hr_attrition_test_df=pd.read_csv('/content/employee_attrition_test.csv')
train_data = hr_attrition_train_df=pd.read_csv('/content/employee_attrition_train.csv')



In [ ]:
train_data.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,50.0,No,Travel_Rarely,1126.0,Research & Development,1.0,2,Medical,1,997,4,Male,66,3,4,Research Director,4,Divorced,17399,6615,9,Y,No,22,4,3,80,1,32,1,2,5,4,1,3
1,36.0,No,Travel_Rarely,216.0,Research & Development,6.0,2,Medical,1,178,2,Male,84,3,2,Manufacturing Director,2,Divorced,4941,2819,6,Y,No,20,4,4,80,2,7,0,3,3,2,0,1
2,21.0,Yes,Travel_Rarely,337.0,Sales,7.0,1,Marketing,1,1780,2,Male,31,3,1,Sales Representative,2,Single,2679,4567,1,Y,No,13,3,2,80,0,1,3,3,1,0,1,0
3,50.0,No,Travel_Frequently,1246.0,Human Resources,NaN,3,Medical,1,644,1,Male,99,3,5,Manager,2,Married,18200,7999,1,Y,No,11,3,3,80,1,32,2,3,32,5,10,7
4,52.0,No,Travel_Rarely,994.0,Research & Development,7.0,4,Life Sciences,1,1118,2,Male,87,3,3,Healthcare Representative,2,Single,10445,15322,7,Y,No,19,3,4,80,0,18,4,3,8,6,4,0


# DATA CLEANING

In [ ]:
#subset the orignal dataset
#its safer to work on copies than original dataframes unless you won't be needing the original df
df5_test_data=test_data.copy()
df5_test_data


,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,NaN,Travel_Rarely,866.0,Sales,NaN,3,Medical,1,1469,4,Male,84,3,2,Sales Executive,1,Single,8463,23490,0,Y,No,18,3,4,80,0,6,4,3,5,4,1,3
1,53.0,Travel_Rarely,1084.0,Research & Development,13.0,2,Medical,1,250,4,Female,57,4,2,Manufacturing Director,1,Divorced,4450,26250,1,Y,No,11,3,3,80,2,5,3,3,4,2,1,3
2,24.0,Travel_Rarely,240.0,Human Resources,22.0,1,Human Resources,1,1714,4,Male,58,1,1,Human Resources,3,Married,1555,11585,1,Y,No,11,3,3,80,1,1,2,3,1,0,0,0
3,45.0,Travel_Rarely,1339.0,Research & Development,7.0,3,Life Sciences,1,86,2,Male,59,3,3,Research Scientist,1,Divorced,9724,18787,2,Y,No,17,3,3,80,1,25,2,3,1,0,0,0
4,36.0,Travel_Rarely,1396.0,Research & Development,NaN,2,Life Sciences,1,304,4,Male,62,3,2,Laboratory Technician,2,Single,5914,9945,8,Y,No,16,3,4,80,0,16,3,4,13,11,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,33.0,Travel_Frequently,430.0,Sales,7.0,3,Medical,1,1477,4,Male,54,3,2,Sales Executive,1,Married,4373,17456,0,Y,No,14,3,1,80,2,5,2,3,4,3,0,3
437,27.0,Non-Travel,210.0,Sales,1.0,1,Marketing,1,449,3,Male,73,3,2,Sales Executive,2,Married,6349,22107,0,Y,Yes,13,3,4,80,1,6,0,3,5,4,1,4
438,NaN,Travel_Rarely,1157.0,Research & Development,2.0,4,Medical,1,440,1,Male,84,1,1,Research Scientist,4,Married,3464,24737,5,Y,Yes,13,3,4,80,0,5,4,2,3,2,2,2
439,39.0,Travel_Rarely,1383.0,Human Resources,2.0,3,Life Sciences,1,909,4,Female,42,2,2,Human Resources,4,Married,5204,7790,8,Y,No,11,3,3,80,2,13,2,3,5,4,0,4


In [ ]:
#subset the orignal dataset
#its safer to work on copies than original dataframes unless you won't be needing the original df
df5_train_data=train_data.copy()
df5_train_data


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,50.0,No,Travel_Rarely,1126.0,Research & Development,1.0,2,Medical,1,997,4,Male,66,3,4,Research Director,4,Divorced,17399,6615,9,Y,No,22,4,3,80,1,32,1,2,5,4,1,3
1,36.0,No,Travel_Rarely,216.0,Research & Development,6.0,2,Medical,1,178,2,Male,84,3,2,Manufacturing Director,2,Divorced,4941,2819,6,Y,No,20,4,4,80,2,7,0,3,3,2,0,1
2,21.0,Yes,Travel_Rarely,337.0,Sales,7.0,1,Marketing,1,1780,2,Male,31,3,1,Sales Representative,2,Single,2679,4567,1,Y,No,13,3,2,80,0,1,3,3,1,0,1,0
3,50.0,No,Travel_Frequently,1246.0,Human Resources,NaN,3,Medical,1,644,1,Male,99,3,5,Manager,2,Married,18200,7999,1,Y,No,11,3,3,80,1,32,2,3,32,5,10,7
4,52.0,No,Travel_Rarely,994.0,Research & Development,7.0,4,Life Sciences,1,1118,2,Male,87,3,3,Healthcare Representative,2,Single,10445,15322,7,Y,No,19,3,4,80,0,18,4,3,8,6,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,NaN,No,Travel_Rarely,750.0,Research & Development,28.0,3,Life Sciences,1,1596,2,Male,46,4,2,Laboratory Technician,3,Married,3407,25348,1,Y,No,17,3,4,80,2,10,3,2,10,9,6,8
1025,41.0,No,Travel_Rarely,447.0,Research & Development,NaN,3,Life Sciences,1,1814,2,Male,85,4,2,Healthcare Representative,2,Single,6870,15530,3,Y,No,12,3,1,80,0,11,3,1,3,2,1,2
1026,22.0,Yes,Travel_Frequently,1256.0,Research & Development,NaN,4,Life Sciences,1,1203,3,Male,48,2,1,Research Scientist,4,Married,2853,4223,0,Y,Yes,11,3,2,80,1,1,5,3,0,0,0,0
1027,29.0,No,Travel_Rarely,1378.0,Research & Development,13.0,2,Other,1,2053,4,Male,46,2,2,Laboratory Technician,2,Married,4025,23679,4,Y,Yes,13,3,1,80,1,10,2,3,4,3,0,3


In [ ]:
df5_train_data.isnull().sum()

Age                         136
Attrition                     0
BusinessTravel                5
DailyRate                    27
Department                    0
DistanceFromHome             95
Education                     0
EducationField                0
EmployeeCount                 0
EmployeeNumber                0
EnvironmentSatisfaction       0
Gender                        0
HourlyRate                    0
JobInvolvement                0
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 5
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                      0
PercentSalaryHike             0
PerformanceRating             0
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC


1.   List item
2.   List item



In [ ]:
#Let’s take a more detailed look at what data is actually missing:
total = df5_train_data.isnull().sum().sort_values(ascending=False)
percent_1 = df5_train_data.isnull().sum()/df5_train_data.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)


,Total,%
Age,136,13.2
DistanceFromHome,95,9.2
DailyRate,27,2.6
BusinessTravel,5,0.5
MaritalStatus,5,0.5


In [ ]:
df5_train_data


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,50.0,No,Travel_Rarely,1126.0,Research & Development,1.0,2,Medical,1,997,4,Male,66,3,4,Research Director,4,Divorced,17399,6615,9,Y,No,22,4,3,80,1,32,1,2,5,4,1,3
1,36.0,No,Travel_Rarely,216.0,Research & Development,6.0,2,Medical,1,178,2,Male,84,3,2,Manufacturing Director,2,Divorced,4941,2819,6,Y,No,20,4,4,80,2,7,0,3,3,2,0,1
2,21.0,Yes,Travel_Rarely,337.0,Sales,7.0,1,Marketing,1,1780,2,Male,31,3,1,Sales Representative,2,Single,2679,4567,1,Y,No,13,3,2,80,0,1,3,3,1,0,1,0
3,50.0,No,Travel_Frequently,1246.0,Human Resources,NaN,3,Medical,1,644,1,Male,99,3,5,Manager,2,Married,18200,7999,1,Y,No,11,3,3,80,1,32,2,3,32,5,10,7
4,52.0,No,Travel_Rarely,994.0,Research & Development,7.0,4,Life Sciences,1,1118,2,Male,87,3,3,Healthcare Representative,2,Single,10445,15322,7,Y,No,19,3,4,80,0,18,4,3,8,6,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,NaN,No,Travel_Rarely,750.0,Research & Development,28.0,3,Life Sciences,1,1596,2,Male,46,4,2,Laboratory Technician,3,Married,3407,25348,1,Y,No,17,3,4,80,2,10,3,2,10,9,6,8
1025,41.0,No,Travel_Rarely,447.0,Research & Development,NaN,3,Life Sciences,1,1814,2,Male,85,4,2,Healthcare Representative,2,Single,6870,15530,3,Y,No,12,3,1,80,0,11,3,1,3,2,1,2
1026,22.0,Yes,Travel_Frequently,1256.0,Research & Development,NaN,4,Life Sciences,1,1203,3,Male,48,2,1,Research Scientist,4,Married,2853,4223,0,Y,Yes,11,3,2,80,1,1,5,3,0,0,0,0
1027,29.0,No,Travel_Rarely,1378.0,Research & Development,13.0,2,Other,1,2053,4,Male,46,2,2,Laboratory Technician,2,Married,4025,23679,4,Y,Yes,13,3,1,80,1,10,2,3,4,3,0,3


Imputation of data

In [ ]:
#imputation with median
missing_rate = ['DailyRate']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_train_data.loc[df5_train_data.loc[:,i].isnull(),i]=df5_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_train_data.isnull().sum()

count of NULL values after imputation



Age                         136
Attrition                     0
BusinessTravel                5
DailyRate                     0
Department                    0
DistanceFromHome             95
Education                     0
EducationField                0
EmployeeCount                 0
EmployeeNumber                0
EnvironmentSatisfaction       0
Gender                        0
HourlyRate                    0
JobInvolvement                0
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 5
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                      0
PercentSalaryHike             0
PerformanceRating             0
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [ ]:
#imputation with median
missing_rate = ['DailyRate']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_test_data.loc[df5_test_data.loc[:,i].isnull(),i]=df5_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_test_data.isnull().sum()

count of NULL values after imputation



Age                         55
BusinessTravel               5
DailyRate                    0
Department                   0
DistanceFromHome            54
Education                    0
EducationField               0
EmployeeCount                0
EmployeeNumber               0
EnvironmentSatisfaction      0
Gender                       0
HourlyRate                   0
JobInvolvement               0
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                3
MonthlyIncome                0
MonthlyRate                  0
NumCompaniesWorked           0
Over18                       0
OverTime                     0
PercentSalaryHike            0
PerformanceRating            0
RelationshipSatisfaction     0
StandardHours                0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInCurrentRole           0
YearsSin

In [ ]:
#imputation with median
missing_rate = ['Age']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_train_data.loc[df5_train_data.loc[:,i].isnull(),i]=df5_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_train_data.isnull().sum()

count of NULL values after imputation



Age                          0
Attrition                    0
BusinessTravel               5
DailyRate                    0
Department                   0
DistanceFromHome            95
Education                    0
EducationField               0
EmployeeCount                0
EmployeeNumber               0
EnvironmentSatisfaction      0
Gender                       0
HourlyRate                   0
JobInvolvement               0
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                5
MonthlyIncome                0
MonthlyRate                  0
NumCompaniesWorked           0
Over18                       0
OverTime                     0
PercentSalaryHike            0
PerformanceRating            0
RelationshipSatisfaction     0
StandardHours                0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInC

In [ ]:
#imputation with median
missing_rate = ['Age']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_test_data.loc[df5_test_data.loc[:,i].isnull(),i]=df5_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_test_data.isnull().sum()

count of NULL values after imputation



Age                          0
BusinessTravel               5
DailyRate                    0
Department                   0
DistanceFromHome            54
Education                    0
EducationField               0
EmployeeCount                0
EmployeeNumber               0
EnvironmentSatisfaction      0
Gender                       0
HourlyRate                   0
JobInvolvement               0
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                3
MonthlyIncome                0
MonthlyRate                  0
NumCompaniesWorked           0
Over18                       0
OverTime                     0
PercentSalaryHike            0
PerformanceRating            0
RelationshipSatisfaction     0
StandardHours                0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInCurrentRole           0
YearsSin

In [ ]:
#imputation with median
missing_rate = ['DistanceFromHome']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_train_data.loc[df5_train_data.loc[:,i].isnull(),i]=df5_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_train_data.isnull().sum()

count of NULL values after imputation



Age                         0
Attrition                   0
BusinessTravel              5
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               5
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [ ]:
#imputation with median
missing_rate = ['DistanceFromHome']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_test_data.loc[df5_test_data.loc[:,i].isnull(),i]=df5_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_test_data.isnull().sum()

count of NULL values after imputation



Age                         0
BusinessTravel              5
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               3
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [ ]:
#imputation with median
missing_rate = ['BusinessTravel']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_test_data.loc[df5_test_data.loc[:,i].isnull(),i]=df5_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_test_data.isnull().sum()

TypeError: ignored

In [ ]:
df5_train_data['BusinessTravel']=df5_train_data['BusinessTravel'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['BusinessTravel']=df5_train_data['BusinessTravel'].cat.codes
df5_train_data.head(10)

In [ ]:
#imputation with median
missing_rate = ['BusinessTravel']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_train_data.loc[df5_train_data.loc[:,i].isnull(),i]=df5_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_train_data.isnull().sum()

count of NULL values after imputation



Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               5
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [ ]:
df5_train_data['MaritalStatus']=df5_train_data['MaritalStatus'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['MaritalStatus']=df5_train_data['MaritalStatus'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_test_data['MaritalStatus']=df5_test_data['MaritalStatus'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['MaritalStatus']=df5_test_data['MaritalStatus'].cat.codes
df5_test_data.head(10)

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37.0,Travel_Rarely,866.0,Sales,8.0,3,Medical,1,1469,4,Male,84,3,2,Sales Executive,1,2,8463,23490,0,Y,No,18,3,4,80,0,6,4,3,5,4,1,3
1,53.0,Travel_Rarely,1084.0,Research & Development,13.0,2,Medical,1,250,4,Female,57,4,2,Manufacturing Director,1,0,4450,26250,1,Y,No,11,3,3,80,2,5,3,3,4,2,1,3
2,24.0,Travel_Rarely,240.0,Human Resources,22.0,1,Human Resources,1,1714,4,Male,58,1,1,Human Resources,3,1,1555,11585,1,Y,No,11,3,3,80,1,1,2,3,1,0,0,0
3,45.0,Travel_Rarely,1339.0,Research & Development,7.0,3,Life Sciences,1,86,2,Male,59,3,3,Research Scientist,1,0,9724,18787,2,Y,No,17,3,3,80,1,25,2,3,1,0,0,0
4,36.0,Travel_Rarely,1396.0,Research & Development,8.0,2,Life Sciences,1,304,4,Male,62,3,2,Laboratory Technician,2,2,5914,9945,8,Y,No,16,3,4,80,0,16,3,4,13,11,3,7
5,34.0,Travel_Rarely,204.0,Sales,14.0,3,Technical Degree,1,666,3,Female,31,3,1,Sales Representative,3,0,2579,2912,1,Y,Yes,18,3,4,80,2,8,3,3,8,2,0,6
6,37.0,Travel_Rarely,144.0,Research & Development,22.0,3,Life Sciences,1,577,4,Male,46,1,1,Laboratory Technician,3,2,4230,19225,0,Y,No,15,3,3,80,0,6,2,3,5,4,4,3
7,39.0,Travel_Rarely,1431.0,Research & Development,1.0,4,Medical,1,332,3,Female,96,3,1,Laboratory Technician,3,-1,2232,15417,7,Y,No,14,3,3,80,3,7,1,3,3,2,1,2
8,45.0,Non-Travel,1052.0,Sales,6.0,3,Medical,1,302,4,Female,57,2,3,Sales Executive,4,2,8865,16840,6,Y,No,12,3,4,80,0,23,2,3,19,7,12,8
9,37.0,Travel_Rarely,1229.0,Research & Development,8.0,1,Life Sciences,1,63,4,Male,36,4,1,Laboratory Technician,4,1,2269,4892,1,Y,No,19,3,4,80,0,1,2,3,1,0,0,1


In [ ]:
#imputation with median
missing_rate = ['MaritalStatus']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_train_data.loc[df5_train_data.loc[:,i].isnull(),i]=df5_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_train_data.isnull().sum()

count of NULL values after imputation



Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [ ]:
#imputation with median
missing_rate = ['MaritalStatus']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df5_test_data.loc[df5_test_data.loc[:,i].isnull(),i]=df5_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df5_test_data.isnull().sum()

count of NULL values after imputation



Age                         0
BusinessTravel              5
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [ ]:
df5_train_data['JobRole']=df5_train_data['JobRole'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['JobRole']=df5_train_data['JobRole'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_train_data['Department']=df5_train_data['Department'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['Department']=df5_train_data['Department'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_train_data['EducationField']=df5_train_data['EducationField'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['EducationField']=df5_train_data['EducationField'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_train_data['Gender']=df5_train_data['Gender'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['Gender']=df5_train_data['Gender'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_train_data['OverTime']=df5_train_data['OverTime'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['OverTime']=df5_train_data['OverTime'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_train_data['Over18']=df5_train_data['Over18'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['Over18']=df5_train_data['Over18'].cat.codes
df5_train_data.head(10)

In [ ]:
X_train = df5_train_data.drop('Attrition', axis=1)
y_train = df5_train_data['Attrition']

X_test = df5_test_data.drop("EmployeeCount", axis=1).copy()

X_train.shape, y_train.shape, X_test.shape


((1029, 34), (1029,), (441, 33))

In [ ]:
df5_train_data['Attrition']=df5_train_data['Attrition'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['Attrition']=df5_train_data['Attrition'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_train_data['Department']=df5_train_data['Department'].astype('category')
#print datatypes - business changes to category
df5_train_data.dtypes
df5_train_data['Department']=df5_train_data['Department'].cat.codes
df5_train_data.head(10)

In [ ]:
df5_test_data['Gender']=df5_test_data['Gender'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['Gender']=df5_test_data['Gender'].cat.codes
df5_test_data.head(10)

In [ ]:
df5_test_data['EducationField']=df5_test_data['EducationField'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['EducationField']=df5_test_data['EducationField'].cat.codes
df5_test_data.head(10)

In [ ]:
df5_test_data['JobRole']=df5_test_data['JobRole'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['JobRole']=df5_test_data['JobRole'].cat.codes
df5_test_data.head(10)

In [ ]:
df5_test_data['Over18']=df5_test_data['Over18'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['Over18']=df5_test_data['Over18'].cat.codes
df5_test_data.head(10)

In [45]:
df5_test_data['OverTime']=df5_test_data['OverTime'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['OverTime']=df5_test_data['OverTime'].cat.codes
df5_test_data.head(10)

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,NaN,Travel_Rarely,866.0,Sales,NaN,3,Medical,1,1469,4,Male,84,3,2,Sales Executive,1,Single,8463,23490,0,Y,0,18,3,4,80,0,6,4,3,5,4,1,3
1,53.0,Travel_Rarely,1084.0,Research & Development,13.0,2,Medical,1,250,4,Female,57,4,2,Manufacturing Director,1,Divorced,4450,26250,1,Y,0,11,3,3,80,2,5,3,3,4,2,1,3
2,24.0,Travel_Rarely,240.0,Human Resources,22.0,1,Human Resources,1,1714,4,Male,58,1,1,Human Resources,3,Married,1555,11585,1,Y,0,11,3,3,80,1,1,2,3,1,0,0,0
3,45.0,Travel_Rarely,1339.0,Research & Development,7.0,3,Life Sciences,1,86,2,Male,59,3,3,Research Scientist,1,Divorced,9724,18787,2,Y,0,17,3,3,80,1,25,2,3,1,0,0,0
4,36.0,Travel_Rarely,1396.0,Research & Development,NaN,2,Life Sciences,1,304,4,Male,62,3,2,Laboratory Technician,2,Single,5914,9945,8,Y,0,16,3,4,80,0,16,3,4,13,11,3,7
5,34.0,Travel_Rarely,204.0,Sales,14.0,3,Technical Degree,1,666,3,Female,31,3,1,Sales Representative,3,Divorced,2579,2912,1,Y,1,18,3,4,80,2,8,3,3,8,2,0,6
6,NaN,Travel_Rarely,144.0,Research & Development,22.0,3,Life Sciences,1,577,4,Male,46,1,1,Laboratory Technician,3,Single,4230,19225,0,Y,0,15,3,3,80,0,6,2,3,5,4,4,3
7,39.0,Travel_Rarely,1431.0,Research & Development,1.0,4,Medical,1,332,3,Female,96,3,1,Laboratory Technician,3,NaN,2232,15417,7,Y,0,14,3,3,80,3,7,1,3,3,2,1,2
8,45.0,Non-Travel,1052.0,Sales,6.0,3,Medical,1,302,4,Female,57,2,3,Sales Executive,4,Single,8865,16840,6,Y,0,12,3,4,80,0,23,2,3,19,7,12,8
9,NaN,Travel_Rarely,1229.0,Research & Development,8.0,1,Life Sciences,1,63,4,Male,36,4,1,Laboratory Technician,4,Married,2269,4892,1,Y,0,19,3,4,80,0,1,2,3,1,0,0,1


In [ ]:
df5_test_data['Department']=df5_test_data['Department'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['Department']=df5_test_data['Department'].cat.codes
df5_test_data.head(10)

In [ ]:
df5_test_data['BusinessTravel']=df5_test_data['BusinessTravel'].astype('category')
#print datatypes - business changes to category
df5_test_data.dtypes
df5_test_data['BusinessTravel']=df5_test_data['BusinessTravel'].cat.codes
df5_test_data.head(10)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_train, y_train) * 100, 2)

print (str(acc_log_reg) + ' percent')


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ValueError: ignored

# Explotaroty Data Analysis



1.found out the shape of the data - how rows and columns do we have
2.   List item



In [ ]:
hr_attrition_train_df.describe().T


,count,mean,std,min,25%,50%,75%,max
Age,893.0,37.930571,9.395978,18.0,31.00,37.0,44.0,60.0
DailyRate,1002.0,800.528942,408.109828,102.0,458.25,801.5,1162.0,1496.0
DistanceFromHome,934.0,9.930407,8.421791,1.0,2.00,8.0,16.0,29.0
Education,1029.0,2.892128,1.053541,1.0,2.00,3.0,4.0,5.0
EmployeeCount,1029.0,1.000000,0.000000,1.0,1.00,1.0,1.0,1.0
EmployeeNumber,1029.0,1024.367347,606.301635,1.0,496.00,1019.0,1553.0,2068.0
EnvironmentSatisfaction,1029.0,2.683188,1.096829,1.0,2.00,3.0,4.0,4.0
HourlyRate,1029.0,66.680272,20.474094,30.0,48.00,67.0,84.0,100.0
JobInvolvement,1029.0,2.713314,0.710146,1.0,2.00,3.0,3.0,4.0
JobLevel,1029.0,2.043732,1.118918,1.0,1.00,2.0,3.0,5.0


In [ ]:
#@title Default title text
#Handling Missing Value
#Check for missing value in any column
hr_attrition_train_df.isnull().sum()

In [ ]:
#Let’s take a more detailed look at what data is actually missing:
total = hr_attrition_train_df.isnull().sum().sort_values(ascending=False)
percent_1 = hr_attrition_train_df.isnull().sum()/hr_attrition_train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)


In [ ]:
df5_train_data['MaritalStatus']=df5_train_data['MaritalStatus'].astype('category')
#print datatypes - marital status changes to category
df5_train_data.dtypes
df5_train_data['MaritalStatus']=df5_train_data['MaritalStatus'].cat.codes
df5_train_data.head(10)

label encoding Gender feature using – scikit-learn library approach


In [ ]:
from sklearn import preprocessing 
label = preprocessing.LabelEncoder() 
 
df5_train_data['Gender']= label.fit_transform(df5_train_data['Gender']) 
print(df5_train_data['Gender'].unique())
df5_train_data

In [ ]:
from sklearn import preprocessing 
label = preprocessing.LabelEncoder() 
 
df5_test_data['Gender']= label.fit_transform(df5_test_data['Gender']) 
print(df5_test_data['Gender'].unique())
df5_test_data

In [ ]:
#subset the orignal dataset
#its safer to work on copies than original dataframes unless you won't be needing the original df
df3=hr_attrition_train_df.copy()
df3.head(10)

In [ ]:
df3.isnull().sum()

In [ ]:
data = [hr_attrition_train_df, hr_attrition_test_df]

for dataset in data:
    mean = hr_attrition_train_df["Age"].mean()
    std = hr_attrition_test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)

    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = hr_attrition_train_df["Age"].astype(int)
hr_attrition_train_df["Age"].isnull().sum()
hr_attrition_test_df["Age"].isnull().sum()


In [ ]:
#print all the data types and thier unique values
for column in hr_attrition_train_df.columns:
  if hr_attrition_train_df[column].dtype==object:
    print(str(column)+':'+str(hr_attrition_train_df[column].unique()))
    print(hr_attrition_train_df[column].value_counts())
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
import missingno as msno
msno.bar(hr_attrition_train_df)


In [ ]:
hr_attrition_train_df


***Training set has 1029 examples and 34 features + te target variable (attrition) 3 of the features are floats, 23 are integer and 9 are objects***

The **BusinessTravel** and **MaritalStatus** has 5 features missing values, which can easily be filled. It will be much more tricky, to deal with the **‘Age’** feature, which has 136 missing values. The **DistanceFromHome** feature needs further investigation it is missing 95, and **DailyRate** feature is missing 27.


Describing data

In [ ]:
hr_attrition_train_df.describe()


In [ ]:
#display both categorical and numerical columns
Categorical_columns=['Attrition','BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
Numerical_columns=['Age','DailyRate','DistanceFromHome','Education','EmployeeCount','EnvironmentSatisfaction','HourlyRate','JobInvolvement','JobLevel','JobSatisfaction','MonthlyIncome','MonthlyRate','NumCompaniesWorked','PercentSalaryHike','PerformanceRating','RelationshipSatisfaction','StandardHours','StockOptionLevel','TotalWorkingYears','TotalWorkingYears','TrainingTimesLastYear','WorkLifeBalance','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager']
print('Categorical_columns  +++++',Categorical_columns)
print('Numerical_columns +++++++++',Numerical_columns)
#hr_attrition_train_df.columns.to_list()


*We* can calculate categorical means for other categorical variables such as EducationField and marital status to get a more detailed sense of our data.


In [ ]:
hr_attrition_train_df.groupby('EducationField').mean()

In [ ]:
hr_attrition_train_df.groupby('Attrition').mean()

In [ ]:
hr_attrition_train_df.groupby('MaritalStatus').mean()

In [ ]:
hr_attrition_train_df.groupby('BusinessTravel').mean()

In [ ]:
hr_attrition_train_df.groupby('Department').mean()

In [ ]:
hr_attrition_train_df.groupby('OverTime').mean()

In [ ]:
hr_attrition_train_df.groupby('JobRole').mean()

In [ ]:
hr_attrition_train_df.groupby('Over18').mean()

In [ ]:
hr_attrition_train_df.groupby('Gender').mean()

# VISUALIZE DATA

In [ ]:
#visualize the data for you to get the first impression of the data


In [ ]:
sns.set(font_scale=1.4)
hr_attrition_train_df['Age'].plot(kind='hist', figsize=(10, 10));
plt.xlabel("Attrition by Age", labelpad=14)
plt.ylabel("Frequency", labelpad=14)
plt.title("Distribution of Attrition by age", y=1.015, fontsize=22);


In [ ]:
#fig,axes = plt.subplots(2,2,figsize=(20,20))
#for idx,cat_col in enumerate(Categorical_columns):
 #   row,col = idx//2,idx%2
  #  sns.countplot(x=cat_col,data=hr_attrition_train_df,hue='Attrition',ax=axes[row,col])

#plt.subplots_adjust(hspace=1)

In [ ]:
fig,ax = plt.subplots(2,3, figsize=(20,20))               # 'ax' has references to all the four axes
plt.suptitle("Distribution of various factors", fontsize=20)
sns.countplot(df3['Attrition'], ax = ax[0,0]) 
sns.countplot(df3['BusinessTravel'], ax = ax[0,1]) 
sns.countplot(df3['Department'], ax = ax[0,2]) 
sns.countplot(df3['EducationField'], ax = ax[1,0])
sns.countplot(df3['Gender'], ax = ax[1,1])  
sns.countplot(df3['OverTime'], ax = ax[1,2]) 
plt.xticks(rotation=20)
plt.subplots_adjust(bottom=0.4)
plt.show()

In [ ]:
#Data Visualization
Gender,MonthlyIncome,Attrition feature

#draw a bar plot of survival by sex
sns.barplot(x="Gender", y="MonthlyIncome", hue="Attrition",data=hr_attrition_train_df)



'''Plots above convey following things about the dataset:
Attrition status: About employees who leave through attrition.
Sex: There are more Men than Women who are employed,at the same time more men than women leave through attrition(approx.)
Martial Status: Single people are more likely to be leave employement through attrition. More married people who are employed probably it's because they have a lot of responsibities.
Over18: Majority of the population over18 are employed are also likely to stay on the job.)'''

In [ ]:
#draw a bar plot of hourlyrate 
ax = sns.heatmap(hr_attrition_train_df.corr(), cmap="coolwarm" annot=True, fmt="d")

flights = hr_attrition_train_df.pivot("HourlyRate", "Education", "MonthyIncome")
ax = sns.heatmap(flights)
plt.title("Heatmap attrition Data")
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot('MonthlyIncome', 'Attrition', data=hr_attrition_train_df, width=0.3, palette="Set2")
plt.xticks(rotation=45)
hr_attrition_train_df['MonthlyIncome'].value_counts()


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot('MonthlyIncome', 'JobRole', data=hr_attrition_train_df, width=0.3, palette="Set2")
plt.xticks(rotation=45)
hr_attrition_train_df['MonthlyIncome'].value_counts()


In [ ]:
import matplotlib.image as mpimg
JG1 = sns.jointplot('Education', 'MonthlyIncome', data=hr_attrition_train_df, kind='scatter', color='g')
JG2 = sns.jointplot('JobLevel', 'MonthlyIncome', data=hr_attrition_train_df, kind='hex', color='b')
JG3 = sns.jointplot('HourlyRate', 'MonthlyIncome', data=hr_attrition_train_df, kind='scatter', color='r')
JG4 = sns.jointplot('JobSatisfaction', 'MonthlyIncome', data=hr_attrition_train_df, kind='hex', color='orange')
JG1.savefig('JG1.png')
plt.close(JG1.fig)
JG2.savefig('JG2.png')
plt.close(JG2.fig)
JG3.savefig('JG3.png')
plt.close(JG3.fig)
JG4.savefig('JG4.png')
plt.close(JG4.fig)
f, ax = plt.subplots(2,2,figsize=(20,16))
ax[0,0].imshow(mpimg.imread('JG1.png'))
ax[0,1].imshow(mpimg.imread('JG2.png'))
ax[1,0].imshow(mpimg.imread('JG3.png'))
ax[1,1].imshow(mpimg.imread('JG4.png'))
[ax.set_axis_off() for ax in ax.ravel()]
plt.tight_layout()


In [ ]:
hr_attrition_train_df.boxplot(by ='Attrition', column =['DailyRate'], grid = False)


In [ ]:

Education = hr_attrition_train_df['Education']
MonthlyIncome = hr_attrition_train_df['MonthlyIncome']
YearsAtCompany = hr_attrition_train_df['YearsAtCompany']
PerformanceRating = hr_attrition_train_df['PerformanceRating']
Age = hr_attrition_train_df['Age']
columns = [Education, MonthlyIncome,YearsAtCompany,PerformanceRating,Age]
fig, ax = plt.subplots(figsize=(10, 5))
ax.boxplot(columns)
plt.show()

In [ ]:
# See the distribution of gender to recognize different distributions
plt.subplots(figsize=(20,4))
sns.countplot(x='Age', data=hr_attrition_train_df);
plt.title('Distribution of Age');


#What Features Are Linked To Employee Attrition?


In [ ]:
plt.subplots(figsize=(20,4))
sns.barplot(data=hr_attrition_train_df, x="Age", y="HourlyRate")


In [ ]:
hr_attrition_train_df['Attrition'].value_counts()

In [ ]:
#visualize number of number of employees who have left the company
plt.subplots(figsize=(18,4))
sns.countplot(x="Age", hue="Attrition",data=hr_attrition_train_df,palette='colorblind')


In [ ]:
#visualizing categorical variables
hr_attrition_train_df['Attrition'].value_counts().plot.bar()


*We access the Age column, call the plot method and pass in hist to the kind argument to output a histogram plot.*


In [ ]:
# boxplot
plt.title('Employee Attrition')
plt.violinplot(hr_attrition_train_df['MonthlyIncome'])
plt.show()

shows the distribution of quantitative data across several levels of one (or more) categorical variables such that those distributions can be compared

In [ ]:
sns.countplot(hr_attrition_train_df['Attrition'])

Since the Attrition is not equally distributed, we obtain bars with unequal heights. The numbers of employees leaving is lower than the ones who remain.



# Predictive Modeling


In [ ]:
df5_test_data

In [ ]:
df500_test_data=test_data.copy()
df500_test_data



In [ ]:
df500_train_data=train_data.copy()

df500_train_data

In [ ]:

from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['Gender']=number.fit_transform(df500_train_data['Gender'].astype('str')) 
df500_test_data['Gender']=number.fit_transform(df500_test_data['Gender'].astype('str')) 
df500_test_data




In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['EducationField']=number.fit_transform(df500_train_data['EducationField'].astype('str')) 
df500_test_data['EducationField']=number.fit_transform(df500_test_data['EducationField'].astype('str')) 
df500_test_data


In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['Department']=number.fit_transform(df500_train_data['Department'].astype('str')) 
df500_test_data['Department']=number.fit_transform(df500_test_data['Department'].astype('str')) 
df500_test_data


In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['BusinessTravel']=number.fit_transform(df500_train_data['BusinessTravel'].astype('str')) 
df500_test_data['BusinessTravel']=number.fit_transform(df500_test_data['BusinessTravel'].astype('str')) 
df500_test_data


In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['JobRole']=number.fit_transform(df500_train_data['JobRole'].astype('str')) 
df500_test_data['JobRole']=number.fit_transform(df500_test_data['JobRole'].astype('str')) 
df500_test_data


In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['OverTime']=number.fit_transform(df500_train_data['OverTime'].astype('str')) 
df500_test_data['OverTime']=number.fit_transform(df500_test_data['OverTime'].astype('str')) 
df500_test_data


In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['Over18']=number.fit_transform(df500_train_data['Over18'].astype('str')) 
df500_test_data['Over18']=number.fit_transform(df500_test_data['Over18'].astype('str')) 
df500_test_data


In [ ]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df500_train_data['MaritalStatus']=number.fit_transform(df500_train_data['MaritalStatus'].astype('str')) 
df500_test_data['MaritalStatus']=number.fit_transform(df500_test_data['MaritalStatus'].astype('str')) 
df500_test_data


In [46]:
#impute missing data
#imputation with median
missing_rate = ['MaritalStatus']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_test_data.loc[df500_test_data.loc[:,i].isnull(),i]=df500_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_test_data.isnull().sum()

#imputation with median
missing_rate = ['MaritalStatus']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_train_data.loc[df500_train_data.loc[:,i].isnull(),i]=df500_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_train_data.isnull().sum()



count of NULL values after imputation

count of NULL values after imputation



Age                         136
Attrition                     0
BusinessTravel                0
DailyRate                    27
Department                    0
DistanceFromHome             95
Education                     0
EducationField                0
EmployeeCount                 0
EmployeeNumber                0
EnvironmentSatisfaction       0
Gender                        0
HourlyRate                    0
JobInvolvement                0
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                      0
PercentSalaryHike             0
PerformanceRating             0
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [ ]:
#imputation with median
missing_rate = ['DailyRate']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_test_data.loc[df500_test_data.loc[:,i].isnull(),i]=df500_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_test_data.isnull().sum()

#imputation with median
missing_rate = ['DailyRate']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_train_data.loc[df500_train_data.loc[:,i].isnull(),i]=df500_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_train_data.isnull().sum()


In [48]:
#imputation with median
missing_rate = ['Age']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_test_data.loc[df500_test_data.loc[:,i].isnull(),i]=df500_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_test_data.isnull().sum()


#imputation with median
missing_rate = ['Age']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_train_data.loc[df500_train_data.loc[:,i].isnull(),i]=df500_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_train_data.isnull().sum()


count of NULL values after imputation

count of NULL values after imputation



Age                          0
Attrition                    0
BusinessTravel               0
DailyRate                    0
Department                   0
DistanceFromHome            95
Education                    0
EducationField               0
EmployeeCount                0
EmployeeNumber               0
EnvironmentSatisfaction      0
Gender                       0
HourlyRate                   0
JobInvolvement               0
JobLevel                     0
JobRole                      0
JobSatisfaction              0
MaritalStatus                0
MonthlyIncome                0
MonthlyRate                  0
NumCompaniesWorked           0
Over18                       0
OverTime                     0
PercentSalaryHike            0
PerformanceRating            0
RelationshipSatisfaction     0
StandardHours                0
StockOptionLevel             0
TotalWorkingYears            0
TrainingTimesLastYear        0
WorkLifeBalance              0
YearsAtCompany               0
YearsInC

In [49]:
#imputation with median
missing_rate = ['DistanceFromHome']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_test_data.loc[df500_test_data.loc[:,i].isnull(),i]=df500_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_test_data.isnull().sum()

#imputation with median
missing_rate = ['DistanceFromHome']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_train_data.loc[df500_train_data.loc[:,i].isnull(),i]=df500_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_train_data.isnull().sum()


count of NULL values after imputation

count of NULL values after imputation



Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [50]:
#imputation with median
missing_rate = ['BusinessTravel']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_train_data.loc[df500_train_data.loc[:,i].isnull(),i]=df500_train_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_train_data.isnull().sum()

#imputation with median
missing_rate = ['BusinessTravel']
 
#Technique 2: Using median to impute the missing values
for i in missing_rate:
 df500_test_data.loc[df500_test_data.loc[:,i].isnull(),i]=df500_test_data.loc[:,i].median()
 
print("count of NULL values after imputation\n")
df500_test_data.isnull().sum()


count of NULL values after imputation

count of NULL values after imputation



Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

In [60]:
X_train = df500_train_data.drop('Attrition', axis=1)
y_train = df500_train_data['Attrition']
X_test =df500_test_data.copy()
X_train.shape, y_train.shape, X_test.shape


((1029, 34), (1029,), (441, 34))

In [61]:
clf = SVC()
clf.fit(X_train, y_train)
y_pred_svc = clf.predict(X_test)
acc_svc = round(clf.score(X_train, y_train) * 100, 2)


print (acc_svc)




82.9


In [62]:

from sklearn.linear_model import LogisticRegression
#Linear Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred_log_reg = clf.predict(X_test)
acc_log_reg = round( clf.score(X_train, y_train) * 100, 2)

print (str(acc_log_reg) + ' percent')


82.99 percent


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [67]:
#K-Nearest Neighbors 
clf = KNeighborsClassifier(n_neighbors = 5)
clf.fit(X_train, y_train)
y_pred_knn = clf.predict(X_test)
acc_knn = round(clf.score(X_train, y_train) * 100, 2)


print (acc_knn)


85.52


In [64]:
#Linear SVM
clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred_linear_svc = clf.predict(X_test)
acc_linear_svc = round(clf.score(X_train, y_train) * 100, 2)


print (acc_linear_svc)


24.2


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [68]:
#Deceision tree
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred_decision_tree = clf.predict(X_test)
acc_decision_tree = round(clf.score(X_train, y_train) * 100, 2)


print (acc_decision_tree)


100.0


In [69]:
#ransom forest
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred_random_forest = clf.predict(X_test)
acc_random_forest = round(clf.score(X_train, y_train) * 100, 2)


print (acc_random_forest)


100.0


In [70]:
#Perceptron
clf = Perceptron(max_iter=5, tol=None)
clf.fit(X_train, y_train)
y_pred_perceptron = clf.predict(X_test)
acc_perceptron = round(clf.score(X_train, y_train) * 100, 2)


print (acc_perceptron)


66.76


In [71]:
#Stochastic Gradient Descent (SGD)
clf = SGDClassifier(max_iter=5, tol=None)
clf.fit(X_train, y_train)
y_pred_sgd = clf.predict(X_test)
acc_sgd = round(clf.score(X_train, y_train) * 100, 2)

print (acc_sgd)


82.7


In [ ]:
#Confusion Matrix


#In our attrition problem case

> Indented block



In [63]:
##First, we train these classifiers with our training data.
##After that, using the trained classifier, we predict the attrition outcome of test data.
##Finally, we calculate the accuracy score (in percentange) of the trained classifier.


In [ ]:
sns.pairplot(df3.iloc[:,[1,29,30,31]], hue='Attrition', size=3.5);

In [ ]:
#SVC,LinearSVC

In [ ]:
#5: Predict on dataset model calculate Accuracy Score
#6: Calculate the model accuracy



In [ ]:
#7: Create a confusion matrix with a Pandas cross table - confusion matrix

from sklearn.metrics import confusion_matrix
tree_clf = DecisionTreeClassifier(max_depth=3,min_samples_leaf = 35)
tree_clf.fit(X_train,y_train)
y_pred = tree_clf.predict(X_test_imp)
print("Test Accuracy: ",accuracy_score(y_test,y_pred))
print("Test F1 Score: ",f1_score(y_test,y_pred))
print("Confusion Matrix on Test Data")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
#8: Print the TN, FN, TP, FP values
# 9: Print the model precision value
# precision is the ratio of  tp / (tp + fp)
#10: # recall = the ratio tp / (tp + fn)
#11: Visualize the confusion matrix with a Heatmap


Predictive model evaluation
results and discussions
Conclusion